### Notes from Yesterday

The Hitchhikers Guide to Python Testing
https://docs.python-guide.org/ 
- testing section - ok
- venv with alias - ok
- decorators with @property - wip
- @patch.object to mock a random value or user input - ok 
- unit test and pytest - ok
- control z to toggle between vim and editor ok 
- ill just set up sublime I think…i dont know, or do i go throoug the little tutorial again??? ugh
- pytest cov - ok
- oh but is his vimrc file in there?? gotta see - screen shot for later
- capfd - ok
- fizz buzz - ok
- parameterize - ok
- TDD test driven development - ok
- type through the guess game and the tests - ok
- testing exceptions - ok
- the use of the dunder methods - wip
- DRY what is (parameterize is DRY) - Dont Repeat your self
- pybite  - ok
- flask api ? - no
- fixtures and the bob bebderos article on pybites - Day 12
- obviously the nice pytest book … already reading Brian Okken - wip
- kind of good b/c havent been using decorator - wip
- fix the sublime tab issues - ok
- this line: guess_str = self.num_guesses == 1 and "guess" or "guesses" - ok
- cant see how to write tests for Labroche and Martingale, maybe as sign code should be refactored?
- finsihed day 11 work will take a break and get out day 12??

In [1]:
# guess.py

import random

MAX_GUESSES = 5
START, END = 1, 20

def get_random_number():
    """Get a random number beween START and END, returns int"""
    return random.randint(START, END)

class Game:
    """Number guess class, make it callable to initiate game"""

    def __init__(self):
        """Init _guesses, _answer, _win to set(), get_random_number(), False"""
        self._guesses = set()
        self._answer = get_random_number()
        self._win = False

    def guess(self):
        """Ask user for input, convert to int, raise ValueError outputting 
           the following when applicable:
           'Please enter a number'
           'Should be a number'
           'Number not in range'
           'Already guessed'
           If all good, return the int"""
        guess = input(f'Guess an number between {START} and {END}: ')
        if not guess:
            raise ValueError('Please enter a number')

        try: 
            guess = int(guess)
        except ValueError:
            raise ValueError('Should be a number')

        if guess not in range(START, END+1):
            raise ValueError('Number not in range')

        if guess in self._guesses:
            raise ValueError('Already guessed')

        self._guesses.add(guess)
        return guess

    def _validate_guess(self, guess):
        """Verify if guess is correct, print the following when applicable:
           {guess} is correct!
           {guess} is too low
           {guess} is too high
           Return a boolean"""
        if guess == self._answer:
            print(f'{guess} is correct!')
            return True
        else: 
            high_or_low = 'low' if guess < self._answer else 'high'
            print(f'{guess} is too {high_or_low}')
            return False

    @property
    def num_guesses(self):
        return len(self._guesses)

    def __call__(self):
        """Entry point / game loop, use a loop break/continue,
           see the tests for the exact win/lose messaging"""

        while len(self._guesses) < MAX_GUESSES:
            try:
                guess = self.guess()
            except ValueError as ve:
                print(ve)
                continue
    
            win = self._validate_guess(guess)
            if win:
                guess_str = self.num_guesses == 1 and "guess" or "guesses"
                print(f'It took you {self.num_guesses} {guess_str}')
                self._win = True
                break
        else:
          # else on while/for = anti-pattern? do find it useful in this case
          print(f'Guess {MAX_GUESSES} times, answer was {self._answer}')

# if __name__ == '__main__':
#     game = Game()
#     game()


In [2]:
game = Game()
game()

Guess an number between 1 and 20: 5
5 is too high
Guess an number between 1 and 20: 3
3 is correct!
It took you 2 guesses


In [10]:
from unittest.mock import patch
import random
import pytest
import ipytest
import ipytest.magics

ipytest.config.rewrite_asserts = True
__file__ = 'Day_011.ipynb'

In [11]:
# test_guess.py

@patch.object(random, 'randint')
def test_get_random_number(m):
    m.return_value = 17
    assert get_random_number() == 17

@patch("builtins.input", side_effect=[11, '12', 'bob', 12, 5,
                                     -1, 21, 7, None])
def test_guess(inp):
    game = Game()
    # good
    assert game.guess() == 11
    assert game.guess() == 12

    # not a number
    with pytest.raises(ValueError):
        game.guess() 

    # already guessed
    with pytest.raises(ValueError):
        game.guess()

    # good
    assert game.guess() == 5

    # value out of range
    with pytest.raises(ValueError):
        game.guess()

    with pytest.raises(ValueError):
        game.guess()

    # good
    assert game.guess() == 7

    # user hit enter without guessing
    with pytest.raises(ValueError):
        game.guess()

def test_validate_guess(capfd):
    game = Game()
    game._answer = 2

    assert not game._validate_guess(1)
    out, _ = capfd.readouterr()
    assert out.rstrip() == '1 is too low'

    assert not game._validate_guess(3)
    out, _ = capfd.readouterr()
    assert out.rstrip() == '3 is too high'

    assert game._validate_guess(2)
    out, _ = capfd.readouterr()
    assert out.strip() == '2 is correct!'

@patch("builtins.input", side_effect=[4, 22, 9, 4, 6])
def test_game_win(inp, capfd):
    game = Game()
    game._answer = 6

    game()
    assert game._win is True

    out = capfd.readouterr()[0]
    expected = ['4 is too low', 'Number not in range', '9 is too high', 
                'Already guessed', '6 is correct!', 'It took you 3 guesses']

    output = [line.strip() for line in out.split('\n') if line.strip()]
    for line, exp in zip(output, expected):
        assert line == exp

@patch("builtins.input", side_effect=[None, 5, 9, 14, 11, 12])
def test_game_lose(inp, capfd):
    game = Game()
    game._answer = 13

    game()
    assert game._win is False
    
ipytest.run()

============================= test session starts ==============================
platform darwin -- Python 3.7.0, pytest-3.8.2, py-1.7.0, pluggy-0.7.1
rootdir: /Users/NewEarth/Documents/Projects/ANewTraining/Python/HundredDaysCode/My100Days/Day_011/ipython, inifile:
plugins: cov-2.6.0
collected 6 items

Day_011.py ......                                                        [100%]

=============================== warnings summary ===============================
/anaconda3/lib/python3.7/site-packages/_pytest/assertion/rewrite.py:268: PytestWarning: Module already imported so cannot be rewritten: pytest_cov
  self.config,

-- Docs: https://docs.pytest.org/en/latest/warnings.html
===================== 6 passed, 1 warnings in 0.09 seconds =====================


In [12]:
def color_wheel():
    red   = set(list(range(1, 10, 2)) + list(range(12, 19, 2)) + list(range(19, 28, 2)) + list(range(30, 37, 2)))
    black = set(list(range(2, 12, 2)) + list(range(11, 18, 2)) + list(range(20, 29, 2)) + list(range(29, 36, 2)))
    green = [37, 38]
    
    spin = random.randrange(1,39)
    
    if spin in green:
        color = 'green'
    elif spin in red:
        color = 'red'
    elif spin in black:
        color = 'black'
    
    return color

def Labouchere(numbers = None):
    # I learned here the pitfalls of trying to use a mutable list as a default variable 
    # def Labouchere(numbers = [1, 2, 3, 4]) led to unexpected results when the fxn was repeatedly evaluated
    # https://docs.python-guide.org/writing/gotchas/
    
    if numbers is None:
        numbers = [1, 2, 3, 4]
    print('numbers = {}'.format(numbers))
    i = 1
    bet_color = 'red'
    winnings = 0
    
    while numbers:
        # check the length of the list
        if len(numbers) < 2:
            bet_amount =  numbers[0]
        else:
            bet_amount = numbers[0] + numbers[-1]
            
        # use the simple wheel
        outcome = color_wheel()
        
        if outcome == bet_color:
            numbers = numbers[1:-1]
            winnings += bet_amount
        else:
            numbers.append(bet_amount)
            winnings -= bet_amount
        
        print('i:{}, bet_amount:{}, outcome:{},   winnings{},    numbers:{}'.format(i, bet_amount, outcome, winnings, numbers))
        i += 1 
        
    return winnings

def Martingale(bet='red'):
    bet_amount = 1
    winnings = 0
    i = 1
    
    while winnings > -100 and winnings < 5:
        outcome = color_wheel()

        if outcome == bet:
            winnings += bet_amount
            bet_amount = 1

        else:
            winnings -= bet_amount
            bet_amount *= 2
            
        print('i:{},\toutcome:{},\twinnings:{},\tbet_amount:{},'.format(i, outcome, winnings, bet_amount))    
        i += 1
    
    return winnings



In [13]:
color_wheel()

'red'

In [18]:
@patch.object(random, 'randrange')
def test_color_wheel(m):
    m.return_value = 9
    assert color_wheel() == 'red'
    m.return_value = 17
    assert color_wheel() == 'black'
    m.return_value = 37
    assert color_wheel() == 'green'
    
ipytest.run('-s')

============================= test session starts ==============================
platform darwin -- Python 3.7.0, pytest-3.8.2, py-1.7.0, pluggy-0.7.1
rootdir: /Users/NewEarth/Documents/Projects/ANewTraining/Python/HundredDaysCode/My100Days/Day_011/ipython, inifile:
plugins: cov-2.6.0
collected 6 items

Day_011.py ......

=============================== warnings summary ===============================
/anaconda3/lib/python3.7/site-packages/_pytest/assertion/rewrite.py:268: PytestWarning: Module already imported so cannot be rewritten: pytest_cov
  self.config,

-- Docs: https://docs.pytest.org/en/latest/warnings.html
===================== 6 passed, 1 warnings in 0.09 seconds =====================


In [21]:
num_guesses = 1
num_guesses == 1 and "guess" or 'guesses'

'guess'

In [22]:
num_guesses = 2
num_guesses == 1 and "guess" or 'guesses'

'guesses'

In [23]:
True and 'guess'

'guess'

In [24]:
False and 'guess'

False

In [25]:
False and 'guess' or 'guesses'

'guesses'